In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['No'].tolist()

总数量：2


[1, 2]

In [3]:
a = 1

a

1

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

output, i = pd.DataFrame(), 0
while True:
    b = 0
    while True:
        b += 1
        print(b, i)
        try:
            resp = requests.get(f'''https://ecatalog.smpcorp.com/V2/STD/api/part/partsearch?filter={dict_param['part']}&filterType={dict_param.get('search', 'n')}&searchType={dict_param.get('type', 'null')}&imageSize=80&start={i*96}&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-''',
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(10, 10))

            if resp.status_code == 200:
                break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    list_json = resp.json()

    # = = = = = = = = = = = = = = =

    len(list_json) == 0
    
    # = = = = = = = = = = = = = = =

    if i == 0:
        count = list_json[0]['maxRows']

    # = = = = = = = = = = = = = = =

    df_temp = pd.DataFrame({'No': [json_['rowId'] for json_ in list_json],
                            'Part Number': [json_['basePart'] for json_ in list_json]})

    output = pd.concat([output, df_temp], ignore_index=True).fillna('')
    
    # = = = = = = = = = = = = = = =

    i += 1

    # = = = = = = = = = = = = = = =

    if len(output) == count:
        output = output.sort_values(by=['No'],
                                    ascending=[True],
                                    ignore_index=True)
        
        output.to_excel('./test_part_number.xlsx', index=False)
        
        break

output

1 0
1 1
1 2


,No,Part Number
0,1,DS-332
1,2,DS-888
2,3,DS-1103
3,4,DS-1104
4,5,DS-1126
...,...,...
213,214,S-2318
214,215,S2344
215,216,S2407
216,217,S2412


In [6]:
crawler_status = 'ok'

crawler_status

'ok'

In [7]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [8]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_number_error.xlsx', index=False)

output_error

""
